In [48]:
import pandas as pd
import os

def substituir_delimitadores(file_path, delimiter_to_replace=';', replacement='|'):
    """Substitui os delimitadores dentro das aspas por um caractere exclusivo."""
    with open(file_path, 'r') as file:
        lines = file.readlines()
        modified_lines = []
        for line in lines:
            within_quotes = False
            modified_line = ''
            for char in line:
                if char == '"':
                    within_quotes = not within_quotes
                if within_quotes and char == delimiter_to_replace:
                    modified_line += replacement
                elif char == "'":
                    modified_line += "''"  # Substitui aspas simples por duas aspas simples
                else:
                    modified_line += char
            modified_lines.append(modified_line)
    return modified_lines

def csv_to_sql(file_path, output_file, table_name, delimiter=';', quotechar='"'):
    """Converte um arquivo CSV para um arquivo SQL."""
    # Substituir delimitadores dentro das aspas
    modified_lines = substituir_delimitadores(file_path, delimiter)
    
    # Salvar as linhas modificadas em um novo arquivo temporário
    temp_file_path = 'temp.csv'
    with open(temp_file_path, 'w') as file:
        file.writelines(modified_lines)
    
    # Carregar o arquivo CSV modificado
    df = pd.read_csv(temp_file_path, delimiter=delimiter, quotechar=quotechar, na_values=[''])  # Tratar campos vazios como vazios
    
    # Remover o arquivo temporário
    os.remove(temp_file_path)
    
    # Substituir barras (/) por sublinhados (_) nos nomes das colunas
    df.columns = df.columns.str.replace('/', '_')
    
    # Abre o arquivo SQL em modo de escrita
    with open(output_file, 'w') as f:
        # Escreve os comandos INSERT INTO no arquivo SQL
        f.write("\n-- Dados\n")
        for index, row in df.iterrows():
            values = ', '.join([f"'{val}'" if pd.notna(val) else "''" for val in row])  # Trata valores nulos corretamente
            f.write(f"INSERT INTO {table_name} VALUES ({values});\n")

# Definir caminhos de entrada e saída e nome da tabela
file_path = 'SP 14 de 14.csv'
output_file = '14.sql'
table_name = 'sp'

# Converter CSV para SQL
csv_to_sql(file_path, output_file, table_name)

print(f"Arquivo SQL gerado em: {output_file}")


C:\Users\testi\AppData\Local\Temp\ipykernel_21436\2908701415.py:35: DtypeWarning: Columns (10,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(temp_file_path, delimiter=delimiter, quotechar=quotechar, na_values=[''])  # Tratar campos vazios como vazios


Arquivo SQL gerado em: 14.sql
